In [1]:
import os
import numpy as np
import pandas as pd

os.chdir("../..")
import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train_lead1 = X_train[:,:,0]
X_test_lead1 = X_test[:,:,0]
X_val_lead1 = X_val[:,:,0]

This experiment has 43 classes


In [4]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential

In [6]:
ann_model = Sequential()
ann_model.add(Dense(50, activation="relu", input_shape=(1000,1)))
ann_model.add(Dense(50, activation="relu"))
ann_model.add(Dense(50, activation="relu"))
ann_model.add(Dense(50, activation="relu"))
ann_model.add(GlobalAveragePooling1D())
ann_model.add(Dense(n_classes, activation="sigmoid"))

In [9]:
ann_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.AUC(num_thresholds=1000000, multi_label=False),
                utils.CustomMetric(beta=0.5)])

In [10]:
history = ann_model.fit(X_train_lead1, y_train, validation_data=(X_val_lead1, y_val), epochs=30)

Epoch 1/30
534/534 [==============================] - ETA: 0s - loss: 0.1279 - auc: 0.9086 - custom_metric: 0.9382

c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric CustomMetric implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


534/534 [==============================] - 119s 191ms/step - loss: 0.1279 - auc: 0.9086 - custom_metric: 0.9382 - val_loss: 0.1180 - val_auc: 0.9226 - val_custom_metric: 0.9469
Epoch 2/30
534/534 [==============================] - 97s 182ms/step - loss: 0.1180 - auc: 0.9221 - custom_metric: 0.9456 - val_loss: 0.1168 - val_auc: 0.9246 - val_custom_metric: 0.9468
Epoch 3/30
534/534 [==============================] - 101s 190ms/step - loss: 0.1156 - auc: 0.9258 - custom_metric: 0.9479 - val_loss: 0.1146 - val_auc: 0.9275 - val_custom_metric: 0.9482
Epoch 4/30
534/534 [==============================] - 118s 221ms/step - loss: 0.1136 - auc: 0.9295 - custom_metric: 0.9502 - val_loss: 0.1127 - val_auc: 0.9306 - val_custom_metric: 0.9509
Epoch 5/30
534/534 [==============================] - 98s 184ms/step - loss: 0.1123 - auc: 0.9315 - custom_metric: 0.9512 - val_loss: 0.1139 - val_auc: 0.9291 - val_custom_metric: 0.9495
Epoch 6/30
534/534 [==============================] - 75s 140ms/step - lo

In [12]:
from sklearn.metrics import roc_auc_score

preds = ann_model.predict(X_test_lead1)

print(roc_auc_score(y_test,preds))
auc = ann_model.evaluate(X_test_lead1, y_test)
print(auc)

68/68 [==============================] - 2s 28ms/step
0.6805787356487152
 2/68 [..............................] - ETA: 5s - loss: 0.0933 - auc: 0.9445 - custom_metric: 0.9614

c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py:2448: UserWarning: Metric CustomMetric implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


68/68 [==============================] - 6s 83ms/step - loss: 0.1117 - auc: 0.9309 - custom_metric: 0.9511
[0.11169098317623138, 0.9308748245239258, 0.9511114358901978]
